
# Bokeh for Time Series Analysis
<hr style="border: 2px solid black;">


<img src="./images/bokeh.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">

<img src="./images/bokeh_at_ag_glance.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">
**Introduction to Bokeh**
Bokeh is an interactive visualization library for Python that targets modern web browsers for presentation.
Unlike Matplotlib, which is primarily designed for static plots, Bokeh excels at creating
interactive plots and dashboards. It can handle large datasets and streaming data,
making it suitable for real-time applications.

**Key Features of Bokeh:**

* **Interactivity:** Built-in support for zooming, panning, hovering, and other interactive tools.
* **Web-Focused:** Generates HTML and JavaScript, making it easy to embed plots in web pages.
* **High Performance:** Can handle large datasets efficiently.
* **Versatility:** Supports a wide range of plot types (lines, bars, scatter plots, etc.).

<hr style="border: 2px solid black;">


**Documentation:**

For comprehensive documentation, please refer to the official Bokeh website: [https://docs.bokeh.org/en/latest/](https://docs.bokeh.org/en/latest/)


<hr style="border: 2px solid black;">


**Lab Exercise:**

Your task is to recreate the time series analysis lab we previously conducted using Pandas,
Matplotlib, and Seaborn, but this time, utilize the Bokeh library for visualization.
This will involve:

1.  Loading and preprocessing the "AirPassengersDates.csv" dataset.
2.  Creating interactive Bokeh plots for:
    * Time series line plots
    * Bar plots of aggregated data
    * Visualizing mean and standard deviation
    * Outlier detection
    * Resampling (upsampling and downsampling)
    * Lag analysis
    * Autocorrelation

Pay close attention to Bokeh's features for interactivity (tools, hover effects) and
its handling of data sources. Aim to replicate the insights and visualizations
from the previous lab while leveraging Bokeh's strengths.

Good luck!
<hr style="border: 2px solid black;">

In [8]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Span
from bokeh.io import output_file
from statsmodels.tsa.stattools import acf

df = pd.read_csv("./datasets/AirPassengersDates.csv")
df["Date"] = pd.to_datetime(df["Date"])
df["Month"] = df["Date"].dt.month
df["Month_Name"] = df["Date"].dt.strftime("%b")
df["Year"] = df["Date"].dt.year

# === Étape 1 : Courbe temporelle avec moyenne et écart-type ===
mean_val = df["#Passengers"].mean()
std_val = df["#Passengers"].std()
source = ColumnDataSource(df)

ts_fig = figure(title="Air Passengers Over Time (Mean & Std Dev)", x_axis_type="datetime", width=800, height=300,
                tools="pan,wheel_zoom,box_zoom,reset,hover,save")
ts_fig.line(x='Date', y='#Passengers', source=source, line_width=2, color="navy")
ts_fig.scatter(x='Date', y='#Passengers', source=source, size=5, color="navy", alpha=0.5)
ts_fig.add_layout(Span(location=mean_val, dimension='width', line_color='red', line_dash='dashed'))
ts_fig.add_layout(Span(location=mean_val + std_val, dimension='width', line_color='green', line_dash='dotted'))
ts_fig.add_layout(Span(location=mean_val - std_val, dimension='width', line_color='green', line_dash='dotted'))
ts_fig.select_one(HoverTool).tooltips = [("Date", "@Date{%F}"), ("#Passengers", "@#Passengers")]
ts_fig.select_one(HoverTool).formatters = {"@Date": "datetime"}

show(ts_fig)

In [9]:
# === Étape 2 : Barplot par mois ===
monthly = df.groupby("Month_Name")["#Passengers"].sum().reset_index()
monthly["Month_Name"] = pd.Categorical(monthly["Month_Name"], 
    categories=["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
                "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"], ordered=True)
monthly = monthly.sort_values("Month_Name")
monthly_source = ColumnDataSource(monthly)

bar_fig = figure(title="Total Passengers per Month", x_range=monthly["Month_Name"].tolist(),
                 width=800, height=300, tools="pan,wheel_zoom,box_zoom,reset,hover,save")
bar_fig.vbar(x="Month_Name", top="#Passengers", width=0.7, source=monthly_source, color="skyblue")

show(bar_fig)

In [10]:
# === Étape 3 : Outliers (Z-score > 2) ===
df["Z_Score"] = (df["#Passengers"] - mean_val) / std_val
outliers = df[df["Z_Score"].abs() > 2]
outliers_source = ColumnDataSource(outliers)

outlier_fig = figure(title="Outlier Detection (Z-Score > 2)", x_axis_type="datetime", width=800, height=300)
outlier_fig.line(df["Date"], df["#Passengers"], line_width=1, color="gray")
outlier_fig.scatter(outliers["Date"], outliers["#Passengers"], size=8, color="red")

show(outlier_fig)

In [12]:
# === Étape 4 : Resampling mensuel (moyenne) ===
resampled = df.set_index("Date").resample("ME")["#Passengers"].mean().reset_index()
resample_source = ColumnDataSource(resampled)
resample_fig = figure(title="Monthly Resampled Time Series", x_axis_type="datetime", width=800, height=300)
resample_fig.line("Date", "#Passengers", source=resample_source, color="green", line_width=2)

show(resample_fig)

In [14]:
# === Étape 5 : Lag plot ===
df["Lag1"] = df["#Passengers"].shift(1)
lag_df = df.dropna()
lag_fig = figure(title="Lag Plot (t vs t-1)", width=400, height=400)
lag_fig.scatter(lag_df["Lag1"], lag_df["#Passengers"], size=6, color="orange", alpha=0.6)
lag_fig.xaxis.axis_label = "t-1"
lag_fig.yaxis.axis_label = "t"

show(lag_fig)

In [15]:
# === Étape 6 : Autocorrelation (ACF) ===
acf_vals = acf(df["#Passengers"], nlags=30)
lags = np.arange(len(acf_vals))
acf_fig = figure(title="Autocorrelation Function", width=800, height=300)
acf_fig.vbar(x=lags, top=acf_vals, width=0.8, color="purple")
acf_fig.xaxis.axis_label = "Lag"
acf_fig.yaxis.axis_label = "ACF"

show(acf_fig)